In [1]:
import os
import sys

os.environ["PYSPARK_PYTHON"] = sys.executable
os.environ["PYSPARK_DRIVER_PYTHON"] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "--master local[4] --executor-memory 4g --driver-memory 4g"

 # Import all necessary libraries
 

In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
from geopy.geocoders import OpenCage
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from opencage.geocoder import OpenCageGeocode
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from opencage.geocoder import OpenCageGeocode
from decimal import Decimal
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import opencage.geocoder
import requests


In [4]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .config("spark.io.compression.codec", "zstd") \
    .config("spark.sql.execution.pythonUDF.arrow.enabled", "true") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sol.shuffle.part-tions", 200) \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.instances", "4") \
    .config("spark.executor.cores", "2") \
    .config("spark.sql.files.maxRecordsPerFile", 10_000) \
    .config("spark.debug.maxToStringFields", 1000) \
    .config("spark.executor.heartbeatInterval", "3600s") \
    .config("spark.network.timeout", "7200s") \
    .config("spark.network.timeoutInterval", "3600s") \
    .enableHiveSupport() \
    .getOrCreate()
 
sc = spark.sparkContext


# Start spark session

In [5]:
spark = SparkSession.builder.appName("SparkETL").getOrCreate()


# Read the restaurants_csv folder

In [6]:
#Прежний метод очень доллго грузился по этому использовался для проверки нагрузки
%load_ext memory_profiler
%memit

folder_path = "restaurant_csv/"

# Получаю список файлов в папке с ресторанами
files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith(".csv")]

# Считайте все CSV-файлы в DataFrame
restaurant_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(files)

#Cхему DataFrame
restaurant_df.printSchema()

#DataFrame
restaurant_df.show()


peak memory: 96.05 MiB, increment: 0.27 MiB
root
 |-- id: long (nullable = true)
 |-- franchise_id: integer (nullable = true)
 |-- franchise_name: string (nullable = true)
 |-- restaurant_franchise_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)

+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|          id|franchise_id|      franchise_name|restaurant_franchise_id|country|          city|   lat|    lng|
+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|197568495625|          10|    The Golden Spoon|                  24784|     US|       Decatur|34.578|-87.021|
| 17179869242|          59|         Azalea Cafe|                  10902|     FR|         Paris|48.861|  2.368|
|214748364826|          27|     The Corner Cafe|                  92040

## Add API key

In [7]:
api_key = '87fd6af7d45949ab92bcead6468a7882'
geocoder = opencage.geocoder.OpenCageGeocode(api_key)

# CHECKING API

In [26]:
# Disable SSL certificate verification (cause of error)
requests.packages.urllib3.disable_warnings()  # Ignore SSL certificate warnings

# test data from first data
lat = 34.578
lon = -87.021

# Construct the URL for the reverse geocoding request
url = f"https://api.opencagedata.com/geocode/v1/json?q={lat}%2C{lon}&key={api_key}"

# Perform the request
response = requests.get(url, verify=False)

# Check if the request was successful
if response.status_code == 200:
    
    # Extract the result
    result = response.json()
    # Get geohash value
    geohash_value = result['results'][0]['annotations']['geohash']
    # Print the geohash value and all data from OpenCage
    print("all:", result)
    print("Geohash:", geohash_value)
else:
    print("Error:", response.status_code)



all: {'documentation': 'https://opencagedata.com/api', 'licenses': [{'name': 'see attribution guide', 'url': 'https://opencagedata.com/credits'}], 'rate': {'limit': 2500, 'remaining': 2499, 'reset': 1707868800}, 'results': [{'annotations': {'DMS': {'lat': "34° 34' 38.56908'' N", 'lng': "87° 1' 14.18304'' W"}, 'FIPS': {'county': '01103', 'state': '01'}, 'MGRS': '16SDD9811026177', 'Maidenhead': 'EM64ln78mn', 'Mercator': {'x': -9687089.591, 'y': 4082346.935}, 'OSM': {'edit_url': 'https://www.openstreetmap.org/edit?way=1225391036#map=17/34.57738/-87.02061', 'note_url': 'https://www.openstreetmap.org/note/new#map=17/34.57738/-87.02061&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=34.57738&mlon=-87.02061#map=17/34.57738/-87.02061'}, 'UN_M49': {'regions': {'AMERICAS': '019', 'NORTHERN_AMERICA': '021', 'US': '840', 'WORLD': '001'}, 'statistical_groupings': ['MEDC']}, 'callingcode': 1, 'currency': {'alternate_symbols': ['US$'], 'decimal_mark': '.', 'disambiguate_symbol': 'US$', 'html_e

In [9]:
# Disable SSL certificate verification (cause of error)
requests.packages.urllib3.disable_warnings()  # Ignore SSL certificate warnings

# test data from first data
lat = 48.861
lon = 2.368

# Construct the URL for the reverse geocoding request
url = f"https://api.opencagedata.com/geocode/v1/json?q={lat}%2C{lon}&key={api_key}"

# Perform the request
response = requests.get(url, verify=False)

# Check if the request was successful
if response.status_code == 200:
    
    # Extract the result
    result = response.json()
    # Get geohash value
    geohash_value = result['results'][0]['annotations']['geohash']
    # Print the geohash value and all data from OpenCage
    print("all:", result)
    print("Geohash:", geohash_value)
else:
    print("Error:", response.status_code)


all: {'documentation': 'https://opencagedata.com/api', 'licenses': [{'name': 'see attribution guide', 'url': 'https://opencagedata.com/credits'}], 'rate': {'limit': 2500, 'remaining': 359, 'reset': 1707782400}, 'results': [{'annotations': {'DMS': {'lat': "48° 51' 39.38040'' N", 'lng': "2° 22' 4.33560'' E"}, 'MGRS': '31UDQ5363612189', 'Maidenhead': 'JN18eu46dp', 'Mercator': {'x': 263590.194, 'y': 6219101.456}, 'NUTS': {'NUTS0': {'code': 'FR'}, 'NUTS1': {'code': 'FR1'}, 'NUTS2': {'code': 'FR10'}, 'NUTS3': {'code': 'FR101'}}, 'OSM': {'note_url': 'https://www.openstreetmap.org/note/new#map=17/48.86094/2.36787&layers=N', 'url': 'https://www.openstreetmap.org/?mlat=48.86094&mlon=2.36787#map=17/48.86094/2.36787'}, 'UN_M49': {'regions': {'EUROPE': '150', 'FR': '250', 'WESTERN_EUROPE': '155', 'WORLD': '001'}, 'statistical_groupings': ['MEDC']}, 'callingcode': 33, 'currency': {'alternate_symbols': [], 'decimal_mark': ',', 'html_entity': '€', 'iso_code': 'EUR', 'iso_numeric': '978', 'name': 'Euro

# Filter out records with null latitude or longitude

In [10]:
# Filter out records with null latitude or longitude
restaurant_df = restaurant_df.filter(restaurant_df["lat"].isNotNull() & restaurant_df["lng"].isNotNull())


In [11]:
restaurant_df.show()

+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|          id|franchise_id|      franchise_name|restaurant_franchise_id|country|          city|   lat|    lng|
+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+
|197568495625|          10|    The Golden Spoon|                  24784|     US|       Decatur|34.578|-87.021|
| 17179869242|          59|         Azalea Cafe|                  10902|     FR|         Paris|48.861|  2.368|
|214748364826|          27|     The Corner Cafe|                  92040|     US|    Rapid City| 44.08|-103.25|
|154618822706|          51|        The Pizzeria|                  41484|     AT|        Vienna|48.213| 16.413|
|163208757312|          65|       Chef's Corner|                  96638|     GB|        London|51.495| -0.191|
| 68719476763|          28|    The Spicy Pickle|                  77517|     US|      Grayling|44.657|-84.744|
|

In [12]:
#DataFrame Schema
restaurant_df.printSchema()

root
 |-- id: long (nullable = true)
 |-- franchise_id: integer (nullable = true)
 |-- franchise_name: string (nullable = true)
 |-- restaurant_franchise_id: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- city: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lng: double (nullable = true)



# Check lat and lng for incorrect values using the OpenCage Geocoding API 

In [25]:
from pyspark.sql.functions import lit, udf
from pyspark.sql.types import StringType
import geohash2

# create udf
def generate_geohash(latitude, longitude):
    return geohash2.encode(latitude, longitude, precision=4)

# register
generate_geohash_udf = udf(generate_geohash, StringType())

# apply
for row in restaurant_df.collect():
    lat = row['lat']
    lon = row['lng']
    geohash = generate_geohash(lat, lon)
    df_with_geohash = restaurant_df.withColumn("geohash", lit(geohash))

# show new df
df_with_geohash.show()


+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+-------+
|          id|franchise_id|      franchise_name|restaurant_franchise_id|country|          city|   lat|    lng|geohash|
+------------+------------+--------------------+-----------------------+-------+--------------+------+-------+-------+
|197568495625|          10|    The Golden Spoon|                  24784|     US|       Decatur|34.578|-87.021|   dnx5|
| 17179869242|          59|         Azalea Cafe|                  10902|     FR|         Paris|48.861|  2.368|   dnx5|
|214748364826|          27|     The Corner Cafe|                  92040|     US|    Rapid City| 44.08|-103.25|   dnx5|
|154618822706|          51|        The Pizzeria|                  41484|     AT|        Vienna|48.213| 16.413|   dnx5|
|163208757312|          65|       Chef's Corner|                  96638|     GB|        London|51.495| -0.191|   dnx5|
| 68719476763|          28|    The Spicy Pickle|

 # Im really trying hard but that error doesn't handle
 